# Evaluating employee performance

## Goals

In this case you will learn how to use aggregate functions to summarize data within a database. You will also develop a greater understanding of the structure of databases and how to use this structure to perform more advanced queries across multiple tables.

## Introduction

**Business Context.** You are an HR representative at a large financial services firm that sells a diverse set of products. In order to make these sales, the firm relies on a call center where sales agents make calls to current as well as prospective customers. The company would like you to analyze their data about the performance of their sales agents and determine who to promote/keep and who to cut.

**Business Problem.** You are tasked with answering the following question: **"Which of our sales agents are the most/least productive?**

**Analytical Context.** The data is split across 3 tables: [`agent.xlsx`](data/agent.xlsx), [`call.xslx`](data/call.xlsx), and [`customer.xlsx`](data/customer.xlsx)

The case is sequenced as follows: you will (1) learn the fundamentals of databases and SQL; (2) use SQL `SELECT` statements to identify potentially interesting customers; and (3) use SQL aggregation functions to compute summary statistics on your agents and identify the most/least productive ones.

## Initializing the SQLite database

Run the following cell to load our database in to the notebook:

**Note:** Don't worry about learning this code, it isn't SQL!

In [ ]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///call_center_database2.db")
%load_ext sql
%sql sqlite:///call_center_database2.db

### Taking a look at the database

The following is an [**entity-relationship digram (ERD)**](https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model) that shows our three tables, their components, and the connections between the tables. Diagrams like the one below are useful for conceptualizing how our database looks from a top-down perspective. As you can see the [primary keys](https://www.w3schools.com/sql/sql_primarykey.ASP) (`AgentID` and `CustomerID`) from the `agent` and `customer` table exist as [foreign keys](https://www.w3schools.com/sql/sql_foreignkey.ASP) on the `call` table. Note the call table also has it's own primary key, `CallID`:

![ERDiagram](data/images/ERDiagram.png)

It's important to know exactly how our tables interact with one another to perform more advanced queries. For instance, SQL uses these connections in order to properly execute `JOIN` operations. Use this diagram as a reference throughout the case to help visualize how our data is structured.

## Using aggregate functions

In order to evaluate our agents' performance, we'll need the help of [**aggregate functions**](https://mode.com/sql-tutorial/sql-aggregate-functions/). An aggregate function allows you to perform a mathematical calculation on a set of values in your database, then returns a single value that summarizes those values in some way. The following are the most commonly-used SQL aggregate functions:

1. **`COUNT()`** - counts rows in a specified column or table
2. **`SUM()`** - calculates the sum of values
3. **`AVG()`** - calculates the average of a set of values
4. **`MIN()`** - finds the minimum value in a set of values
5. **`MAX()`** - finds the maximum value in a set of values

In order to get a general idea of how our agents are performing, let's get a tally of how many calls were made vs. the total products sold. We'll be using `COUNT()` to get the total number of rows in the calls table and `SUM()` to add up all the 1s in the `ProductSold` column:

In [ ]:
%%sql

SELECT COUNT(*) AS TotalCalls, SUM(ProductSold) AS TotalSales 
FROM call

Note that we used `COUNT(*)` in this example. `COUNT(*)`, like `SELECT *`, can be read as "Count all" and counts every row in a table without needing to specify a column.

### Evaluating our agents individually

It looks like our agents have a ~21% success rate as a group, but this doesn't tell us anything about their individual performance. To look at success rate on an agent-by-agent basis, we'll need a new keyword: **`GROUP BY`**. [GROUP BY](https://www.w3schools.com/sql/sql_groupby.asp) combines rows that have the same value in a particular column and is often used in conjunction with aggregate functions to perform multiple mathematical operations at once. The syntax is:
~~~sql

GROUP BY column_name

~~~

where the rows will be grouped by identical values within the column specified. For instance, using `GROUP BY Occupation` on the customer table will create a single row for each unique occupation. Any aggregate function will then be applied on the values in the rows that were grouped together for a particular occupation.

`GROUP BY` is used after `SELECT` and any conditionals such as `WHERE` or `CASE...END`, but before `ORDER BY`.

### Exercise 1

Write a query that outputs each agent's name, their total calls made, and their total successful sales. Order the new query by agent name in ascending order.

**Answer.**

-------

We could now divide the number of successful sales by the total number of calls for each agent to determine each agent's success rate, but there's an easier way to get this information with the `AVG()` function. Since our `ProductSold` column is either a 0 or 1 taking the average will give us the percentage success rate for each of our agents. Let's try using `AVG()` on the `ProductSold` column and order this new column from largest to smallest (descending):

In [ ]:
%%sql

SELECT agent.name, COUNT(*) AS TotalCalls, AVG(call.ProductSold) AS SuccessPercent FROM call
JOIN agent ON agent.AgentID = call.AgentID
GROUP BY agent.name
ORDER BY SuccessPercent DESC

We can see that in terms of percentage of calls that end in a sale, our highest performing agent is Gloria Singh and our lowest performing agent is Angel Briggs. From our earlier query we know the success rate is ~21%. Most of our agents fall within ~1% of this average with Angel Briggs being an obvious outlier.

### Exercise 2

Write a query to find the average, minimum, and maximum call duration for *each* of our agents. Include a conditional to ensure that only calls with durations greater than 0 are used.

**Answer.**

-------

While our `SuccessPercent` metric is useful, it may not be entirely fair to judge our agents' performance entirely based on this. For instance, this metric doesn't consider whether a call was even answered or not, nor does it consider the age of the customer answering. 

### Exercise 3

Modify the query in Exercise 2 to only consider calls which the customer answered, and where the customer's age is at least 18.

**Hint:** You may want to use the **`AND`** and **`OR`** keywords.

**Answer.**

-------

## The `IN` keyword

Another useful tool for creating conditional statements (and a quick shorthand for the `OR` keyword) is the keyword **`IN`**. [IN](https://mode.com/sql-tutorial/sql-in-operator/) is used in the following way:

~~~sql
SELECT * FROM customer
WHERE age IN (18,20,25,30,35)
~~~

This would select any records where the age was 18 ,20 ,25 ,30 *or* 35.

`IN` can also use a `SELECT` statement as a parameter. For instance,

~~~sql
SELECT * FROM call
WHERE CustomerID IN (SELECT CustomerID FROM customer)
~~~

This would first select all of the values in the `CustomerID` column from the `customer` table and then uses that as a parameter to select all of the rows from the `call` table where one of those `CustomerID` values is present. This is especially useful if you have a column with values from multiple tables and would like your query to only select for ones that come from a single table.

In the case of our `call` table, it should only have `AgentID` values that match values from the `agent` table, but we can use the `IN` keyword to make sure. We ran a query at the start that showed we have 9,940 calls in our `call` table. Let's now write a query using `IN` to make sure all of the values in the `AgentID` column of the `call` table has a match in the `agent` table:

In [ ]:
%%sql

SELECT COUNT(*) FROM call
WHERE AgentID IN (SELECT AgentID FROM agent)

Notice this appears to be 1 short of the number we are looking for. Let's run a clean query of the table without any conditionals to confirm:

In [ ]:
%%sql

SELECT COUNT(*) FROM call

This confirms there may be an issue with our database. A single row in the `call` table appears to not have a valid `AgentID`. If we sort the `AgentID` column by ascending values, we can see that our first row has a "-1" in the `AgentID` column. There is no corresponding "-1" on the `agent` table so this may be a mistake. 

In a real-life context, we would probably investigate further as to why this row exists and whether it needs to be deleted or modified. We would also want to re-assess the relational integrity amongst our tables. For now though, we can see the usefulness of the `IN` function for performing more advanced conditional statements.

### Exercise 4

Based on our `SuccessPercent` metric, we currently have a list of our top 5 employees in terms of sales:


| Agent         | AgentID   |
| -----------   | ----------|
| Todd Morrow   |     3     |
| Gloria Singh  |     6     |
| Dana Hardy    |     9     |
| Jocelyn Parker|     1     |
| Agent X       |     10    | 

Use what you've learned so far to write two queries, one for our top five agents and another for the bottom six agents. Each query should show the agent's name, the total number of calls they made, and their product sales conversion percentage. Filter each query to only include calls with customers who are 35 or older, and whose job contains the word "Engineer".

**Answer.**

-------

## Aliasing and the implicit `JOIN`

We've learned before how to assign a new name to a column in a query using the `AS` keyword. This operation is known as an **alias** and is useful for more than just tidying up the result of your queries (read more about aliases [here](https://www.w3schools.com/sql/sql_alias.asp)). One functional use of an alias is in what's called an *implicit* `JOIN`, which is a join statement that doesn't use the `JOIN` keyword at all. Instead it uses aliases to identify which table to query the requested information from. The syntax for an implicit `JOIN` is as follows:

~~~sql
SELECT a.column_from_table_a, b.column_from_table_b
FROM table_a AS a, table_b AS b
WHERE a.shared_column = b.shared_column
~~~

As you can see, we defined the alias for each of our tables in the `FROM` statement, and then referenced the alias in our requested columns by using a `.` in the `SELECT` statement, referred to as **dot notation.** We still use our `WHERE` statement to say on what column our tables are to be joined.

By putting both our requested tables in the `FROM` statement and then using our aliases, SQL can infer that we intend to `JOIN` these tables without needing to use the specific keyword.

### Example 1

Use an implicit `JOIN` to find the average duration of calls for *each* `agent` in the `call` table.

**Answer.** Shown below:

In [ ]:
%%sql

SELECT a.name, AVG(b.Duration)
FROM agent a, call b
WHERE a.AgentID = b.AgentID
GROUP BY a.name

Note in this case we don't even need to use the `AS` keyword to create an alias. Using `AS` can make your code clearer for others to read, but is optional and the code will function the same either way.

## Subqueries

In our discussion of the `IN` keyword, we saw how we could use a set of values *or* another query within a conditional statement. Using another query within a query is formally known as a **subquery**. A [subquery](https://www.w3resource.com/sql/subqueries/understanding-sql-subqueries.php) isn't just a useful time-saver; it also helps maintain the integrity of your queries by making your queries *relational* to other values in your tables.

A subquery can be a part of your `SELECT` or `FROM` statements, or as a conditional in a `WHERE` statement. It must be enclosed in parentheses and can include most of the operators of a normal query, the most notable exception being `ORDER BY`. Below is an example of a subquery for our current dataset:

~~~sql

SELECT name, CustomerID, age FROM customer
WHERE age >= (SELECT AVG(age) FROM customer)

~~~

A subquery first executes the inner query:

In [ ]:
%%sql

SELECT AVG(age) FROM customer

It then uses this value to perform the outer query. We can plug in the number that results from the inner query into our outer query to see our result without explicitly using a subquery:

In [ ]:
%%sql

SELECT name, CustomerID, age FROM customer
WHERE age >= 24.435
LIMIT 15

It turns out that at this moment in time, this will give the same result. However, using a constant in place of a dynamic subquery creates significant problems down the line. For example, if even a single value in the `age` column of our `customer` table changes, that constant will no longer be valid and our outer query will return an incorrect result. By using a subquery, we ensure that the entire query remains *relational* to our dataset and maintains integrity even when changes are made to the database.

### Exercise 5

Use a subquery to get the name, call ID, and duration for every call where the duration is greater than the average call duration. Make sure to *exclude* calls with a `Duration` of 0 from the average calculation. Limit the results to the first 15 calls.

**Answer.**

-------

## Conclusions & Takeaways

In this case, you used aggregate functions to evaluate employee performance and learned how to perform more advanced queries using multiple joins and complex conditional logic. You developed a better understanding of the structure of a database and how the data amongst multiple tables connects and interacts. You also learned how to use aliases, implicit `JOIN`s, and subqueries.

## Appendix

**Aggregate Functions**

- `COUNT()` - counts rows in a specified column or table 
- `SUM()` - calculates the sum of values 
- `AVG()` - calculates the average of a set of values 
- `MIN()` - gets the minimum value in a set of values 
- `MAX()` - gets the maximum value in a set of values

**New Functions**

- `GROUP BY` - Combines rows with identical values, often used in combination with aggregate functions

~~~sql
SELECT SUM(value) FROM table
GROUP BY value

~~~

- `WHERE ... IN` - Used as shorthand for multiple `OR` conditionals

~~~sql
WHERE column_name IN (value1, value2, ...)
WHERE column_name IN (SELECT query)

~~~

- Alias - Created using the optional `AS` keyword, used in subqueries and implicit joins

~~~sql

SELECT a.column_from_table_a, b.column_from_table_b
FROM table_a AS a, table_b AS b

~~~

- Implicit `JOIN` - A joy using alias' that doesn't require the `JOIN` keyword

~~~sql

SELECT a.column_from_table_a, b.column_from_table_b
FROM table_a AS a, table_b AS b
WHERE a.shared_column = b.shared_column

~~~

- Subquery - A query used within another query to helps maintain relational integrity of more complex queries

~~~sql

SELECT column FROM table
WHERE (Select value FROM table)     

~~~
